In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

os.getcwd()
os.chdir('/Users/ting/Desktop/kaggle_Digit_Recognizer/')

#read data
x = pd.read_csv("train.csv")
x 
x_train = np.array(x.loc[:, 'pixel0' : 'pixel783'])
x_train = x_train.astype(np.float32)
x_train.shape

#改變y_train的資料型態
#y_train = np.array(x.loc[:, 'label'])
y_train = pd.DataFrame(x.loc[:, 'label'])
y_1 = []
for i in range(0, 42000):
    lab_nu = y_train.loc[:, 'label'][i]
    y = np.zeros(10)
    y[lab_nu] = 1
    y = [y]
    y_1.append(y)

y_1 = np.array(y_1)

#reshape
y_2 = []
for i in range(0, 42000):
    y_3 = y_1[i][0]
    y_2.append(y_3)

y_2 = np.array(y_2)
y_2.shape
y_train_1 = y_2
y_train = y_train_1.astype(np.float32)

x_test = pd.read_csv("test.csv").astype(np.float32)
x_test = np.array(x_test.loc[:, 'pixel0' : 'pixel783'])
x_test = x_test.astype(np.float32)
x_test
x_test.shape

(28000, 784)

In [ ]:
#set parameters
n_features = x_train.shape[1]
n_labels = y_train.shape[1]

#set input/output
sess = tf.InteractiveSession()
with tf.name_scope('Input'):
    x = tf.placeholder(tf.float32, shape = [None, n_features])
with tf.name_scope('Labels'):
    y = tf.placeholder(tf.float32, shape = [None, n_labels])

#def varible
def weight_variable(shape):
    initial = tf.truncated_normal(shape = shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

#def conv and maxpool
def conv2d(x, w):
    return tf.nn.conv2d(x, w, strides = [1,1,1,1], padding = 'SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

#set layer
#first
with tf.name_scope('FirstConvolutionLayer'):
    wei_conv_1 = weight_variable([5,5,1,32])
    bia_conv_1 = bias_variable([32])
    x_image = tf.reshape(x, [-1,28,28,1])
#tf.reshape(x, [-1???,28,28,1])
    conv_1 = tf.nn.relu(conv2d(x_image, wei_conv_1) + bia_conv_1)
    max_pool_1 = max_pool_2x2(conv_1)

#sec
with tf.name_scope('SecConvLayer'):
    wei_conv_2 = weight_variable([5,5,32,64])
    bia_conv_2 = bias_variable([64])
    conv_2 = tf.nn.relu(conv2d(max_pool_1, wei_conv_2) + bia_conv_2)
    max_pool_2 = max_pool_2x2(conv_2)
    
#full connection
with tf.name_scope('FullConnectionLayer'):
    wei_conv_3 = weight_variable([7 * 7 *64, 1024])
    bia_conv_3 = bias_variable([1024])
    max_pool_2_flat = tf.reshape(max_pool_2, [-1, 7 * 7 * 64])
    fcon_1 = tf.nn.relu(tf.matmul(max_pool_2_flat, wei_conv_3) + bia_conv_3)

#dropout fun
with tf.name_scope('DropFun'):
    keep_prob = tf.placeholder(tf.float32)
    fcon_1_dop_fun = tf.nn.dropout(fcon_1, keep_prob = keep_prob)

#outlayer
with tf.name_scope('OutLayer'):
    wei_out = weight_variable([1024, 10])
    bia_out = bias_variable([10])
    y_outlay = tf.matmul(fcon_1_dop_fun, wei_out) + bia_out

#train and opt
with tf.name_scope('CossEntropy'):
    cross_entro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = y_outlay))
#labels = y, logits = y_outlay
    tf.summary.scalar("CossEntropy", cross_entro)
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entro)
#tf.train.AdamOptimizer(1e-4???).minimize(cross_entro)
correct_pre = tf.equal(tf.argmax(y_outlay, 1), tf.argmax(y, 1))
with tf.name_scope('Accurancy'):
    acc = tf.reduce_mean(tf.cast(correct_pre, tf.float32))
    tf.summary.scalar("Accurancy", acc)

#init
init = tf.global_variables_initializer()
sess.run(init)

#visualization
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('TensorBoard/', graph = tf.get_default_graph())

#opt  
tra = []
for i in range(15000):
    ran = np.random.randint(1, 42000, 50)
    x_tra = x_train[ran]
    y_tra = y_train[ran]
#    batch = mnist.train.next_batch(50)
#batch[1], batch[0] == batch_xs, batch_ys
    if i%100 == 0:
        train_acc = acc.eval(feed_dict = {x: x_tra, y: y_tra, keep_prob: 1.0})
        print(i, train_acc)
#        summary = sess.run(merged, feed_dict = {x: x_tra, y: y_tra, keep_prob: 1.0})
# x feed不進去？
#        writer.add_summary(summary, i)
#write.add_summary(summary, i???)
#        writer.flush()
    train_step.run(feed_dict = {x: x_tra, y: y_tra, keep_prob: 0.5})
#train_step.run again???

res_1 = y_outlay.eval(feed_dict = {x: x_train, keep_prob: 1.0})
res_1 = pd.DataFrame(res_1)
#test
#print(acc.eval(feed_dict = {x: x_test, y: y_test, keep_prob: 1.0}))
#print(y_outlay.eval(feed_dict = {x: x_test, keep_prob: 1.0}))
#res = y_outlay.eval(feed_dict = {x: x_test, keep_prob: 1.0})
res = y_outlay.eval(feed_dict = {x: x_test, keep_prob: 1.0})
#type(res)

#res = tf.argmax(res, 1)
#res = np.nonzero(res)[1]
res = pd.DataFrame(res)

sess.close()

res_1.to_csv("res_1.csv")
res.to_csv("res.csv")

0 0.1
100 0.86
200 0.94
300 0.92
400 0.88
500 0.98
600 0.96
700 0.98
800 0.86
900 0.94
1000 0.96
1100 0.96
1200 1.0
1300 0.98
1400 0.96
1500 0.98
1600 0.96
1700 0.94
1800 0.94
1900 0.94
2000 0.94
2100 0.96
2200 0.98
2300 0.98
2400 1.0
2500 1.0
2600 0.98
2700 1.0
2800 1.0
2900 0.96
3000 0.94
3100 0.96
